In [2]:
import lasagne
import numpy as np
import theano
import theano.tensor as T

dtype = theano.config.floatX
ndim_x = 10

class KernelLayer(lasagne.layers.Layer):
    def __init__(self, incoming, B=lasagne.init.Normal(0.01), **kwargs):
        super(KernelLayer, self).__init__(incoming, **kwargs)
        num_inputs = self.input_shape[1]
        self.eye = T.eye(num_inputs)
        self.B = self.add_param(B, (num_inputs, ), name='B')

    def get_output_for(self, input, **kwargs):
        D = T.dot(self.B*self.eye, self.B*self.eye.T)
        inner = (input.dot(D)*input).sum(axis=1)
        return T.exp(-inner)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0],)

input_var = T.fmatrix('inputs')
target_var = T.fvector('targets')

l_in = lasagne.layers.InputLayer(shape=(None, ndim_x),
                                input_var=input_var)
l_dot = KernelLayer(l_in, name='kernel_layer')

prediction = lasagne.layers.get_output(l_dot)

loss = (prediction-target_var)**2
loss = loss.mean()

params = lasagne.layers.get_all_params(l_dot, trainable=True)

updates = lasagne.updates.adam(
            loss, params, learning_rate=0.01)

train_fn = theano.function([input_var, target_var], [loss, prediction], updates=updates,
                            on_unused_input='ignore')

M = 2   # batch dimension
x_data = 0.5*np.ones((M, ndim_x)).astype(np.float32)
xo_data = np.zeros((M, ndim_x)).astype(np.float32)
target_data = np.ones(M).astype(np.float32)

for i in range(1000):
    train_err, pred = train_fn(x_data-xo_data, target_data)
    if i % 100 == 0:
        print(train_err)
    
print(params[0].get_value())

9.93443336483324e-08
5.115907697472721e-13
3.197442310920451e-14
1.4210854715202004e-14
3.552713678800501e-15
3.552713678800501e-15
3.552713678800501e-15
0.0
0.0
0.0
[ -1.67152479e-08   1.57049669e-06   3.73972252e-05   7.00563896e-06
  -3.34702665e-04  -1.58101807e-06   8.89310243e-08   7.86923593e-08
   3.64859479e-05   7.09714641e-07]
